In [1]:

#### 最初に必要なライブラリを読み込みます。
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit, QubitBra, measure_all, measure_all_oneshot,measure_partial, matrix_to_qubit
from sympy.physics.quantum.gate import H,X,Y,Z,S,T,CPHASE,CNOT,SWAP,UGate,CGateS,gate_simp,OneQubitGate
from sympy.physics.quantum.gate import IdentityGate as _I
from sympy.physics.quantum.qft import *
from sympy.physics.quantum.matrixcache import matrix_cache
matrix_cache.cache_matrix('Tdg',Matrix([[1, 0], [0, exp(-I*pi/4)]]))
matrix_cache.cache_matrix('Sdg',Matrix([[1, 0], [0, -I]]))
matrix_cache.cache_matrix('V',(1/2)*Matrix([[1+I, 1-I], [1-I, 1+I]]))
matrix_cache.cache_matrix('Vdg',(1/2)*Matrix([[1-I, 1+I], [1+I, 1-I]]))

CZ = CPHASE

class Tdg(OneQubitGate): # T^{\dagger} 演算
    gate_name = u'Tdg'
    gate_name_latex = u'T^{\dagger}'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('Tdg', format)

class Sdg(OneQubitGate): # S^{\dagger} 演算
    gate_name = u'Sdg'
    gate_name_latex = u'S^{\dagger}'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('Sdg', format)

class V(OneQubitGate): # √X 演算
    gate_name = u'V'
    gate_name_latex = u'V'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('V', format)

class Vdg(OneQubitGate): # √X ^{\dagger}演算
    gate_name = u'Vdg'
    gate_name_latex = u'V^{\dagger}'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('Vdg', format)

def CV(c,t):  return CGateS((c),V(t))
def CVdg(c,t):  return CGateS((c),Vdg(t))

def CCX(c1,c2,t):  return CGateS((c1,c2),X(t))
def Toffoli(c1,c2,t):  return CGateS((c1,c2),X(t))
def CCZ(c1,c2,t): return (H(t)*CCX(c1,c2,t)*H(t)) # CCZ演算子を定義します。
def hadamard(s,n):
    h = H(s)
    for i in range(s+1,n+s): h = H(i)*h
    return h

def disp1Q(u): print(u); display(represent(u,nqubits=1)); CircuitPlot(u,nqubits=1)
def disp2Q(u): print(u); display(represent(u,nqubits=2)); CircuitPlot(u,nqubits=2)

from sympy.printing.dot import dotprint
init_printing()

%matplotlib inline
import matplotlib.pyplot as plt
from sympy.physics.quantum.circuitplot import CircuitPlot,labeller, Mz,CreateOneQubitGate
alpha, beta, gamma, delta = Symbol(r'\alpha'), Symbol(r'\beta'), Symbol(r'\gamma'), Symbol(r'\delta'), 
psi, phi, theta, chi = Symbol(r'\psi'), Symbol(r'\phi'), Symbol(r'\theta'), Symbol(r'\chi') 
from qutip import *
import numpy as np


（副読本）
* ニールセン・チャン「量子コンピュータと量子通信」（ここでは N,C"QCQI" と書きます）
* arXivに公開されている論文

## 4.4 量子コンピュータと機械学習


（前提知識）機械学習の問題設定

* 教師あり学習

 入出力がペアになったデータから、入出力の関数関係を学習します。
 未来の入力に対する出力を予測する情報処理技術として重要。
 

* 教師なし学習

 一般のデータから本質的な構造を取り出し、理解しやすい解釈を与えて、科学的な発見に結びつけることを重視する傾向がある技術です。


##### 教師あり学習

* 判別問題

 迷惑メール判定　／　文字認識
 

* 回帰分析

 入力 $x$ 、出力 $y$ をペアでデータとして扱い、$x \in \mathcal{X} $ に対する $ y \in \mathbb{R} $ をできるだけ精度よく予測する
 
 - 線形モデル
 - 非線形モデル … 例）ニューラルネットワークモデル


##### 教師なし学習

* 特徴抽出

* 分布推定








#### 線形回帰モデル

入力 $ \boldsymbol{x} = (x_1, x_2, \dots , x_d) \in \mathbb{R} ^{d} $ に対する出力 $ y \in \mathbb{R} $ を

$ y = \theta_{0} + \theta_{1} \phi _{1} (\boldsymbol{x}) + \theta_{2} \phi _{2} (\boldsymbol{x}) + \cdots + \theta_{D} \phi _{D} (\boldsymbol{x}) + \varepsilon $

のようにモデリングします。ここで、$\theta_{0} , \theta_{1} , \theta_{2} , \dots , \theta_{D} $ は関数形を決めるパラメータです。また、誤差 $ \varepsilon $ のズレを許容します。

$ \phi _{1} (\boldsymbol{x}) ,  \phi _{2} (\boldsymbol{x}) , \dots ,  \phi _{D} (\boldsymbol{x}) $ は適当な基底関数です。



#### サポートベクトルマシン（SVM）

サポートベクトルマシンは、２値判別のための代表的な学習アルゴリズムです。次のような特徴があります。

1. マージン最大化基準による学習：データ点から判別境界までの距離を大きくする
2. 凸２次計画問題としての定式化：パラメータ推定のために効率的な計算アルゴリズムを利用可能
3. カーネル関数を使ったモデリング：高い表現能力を持つモデルで学習を行う

２値ラベルをもつデータ $(x_1, y_1), \dots (x_{n}, y_{n}) $  $ ( x_i \in \mathbb{R} ^{d} , y_i \in \{+1, -1\} )$ から線形判別器を学習することを考えます。

線形判別器のモデルを

$ \mathcal{H} = \{  sign ( f(x) ) \,  \vert \, f(x) = \mathbf{w} ^{T} \mathbf{x} + b ,  \mathbf{w} \in \mathbb{R} ^{d} , b \in \mathbb{R} \}$

とします。

ある線形判別器 $ h (\mathbf{x} ) = sign (  \mathbf{w} ^{T} \mathbf{x} + b )$ に対して

$ h(x_i) = y_i  \, \, \, ( \forall i = 1, \dots , n ) $

が満たされるとき、その学習データは線形分離可能といいます。このとき次が成り立ちます。

$ y_i ( \mathbf{w} ^{T} \mathbf{x}_{i} + b ) \gt 0 \, \, \,  ( i = 1, \dots , n )$

#### 主成分分析

$ d $ 次元データ $ x_1, \dots , x_n $ が観測されたとき、散らばり具合をできるだけ保ったままデータの低次元の表現を得ることを考えます。

**主成分分析（principal component analysis; PCA）** では、$d$ 次元データをそれより次元の低い $k$ 次元平面 $ W $ に線形変換します。

$ W $ は原点を通る超平面 $ W _{0} $ とベクトル $ \mathbf{a} \in \mathbb{R} ^d $ を用いて、$ W = W _0 + \mathbf{a} $ と表せます。

部分空間 $ W_ 0 $ への射影行列を $ \Pi $ とすると、データ点 $ \mathbf{x} $ は $ W $ への射影により

$ x \longmapsto \mathbf{a} + \Pi ( \mathbf{x} -  \mathbf{a} ) $

と変換されます。


特異値分解： https://qiita.com/kidaufo/items/0f3da4ca4e19dc0e987e

### 4.4.4 量子線形回帰

損失関数（２乗誤差）を最小化する問題

　↓
 
$ X ^{T} X \mathbf{w} = X ^{T} \mathbf{y}  $

最適なパラメータは $ \mathbf{w} ^{*} = ( X ^{T} X ) ^{-1} X ^{T} \mathbf{y} $ で計算できます。

$ ( X ^{T} X ) ^{-1} X ^{T} $ は、$ X $ の擬似逆行列と呼ばれ、行列 $ X $ が、

$ X = U diag( \sigma_1 \cdots \sigma_{M} ) V ^{T} $ のように特異値分解により低ランク近似できると仮定して

$ ( X ^{T} X ) ^{-1} X ^{T}  = V diag \left(  \frac{1}{\sigma_1} , \cdots , \frac{1}{\sigma_{M}}  \right) U ^{T} $


量子版・・・

（学習）

１）補助量子ビット $ \lvert + \rangle _ {a} $ を用意し、$ \lvert 0 \rangle _ {a} $ のとき以下の操作を行う。

２） $ \displaystyle \frac{1}{\sqrt{N}} \sum _ {i = 1 } ^ {N} \lvert x _{i} \rangle \lvert i \rangle $ を用意する。

３）行列 $ X ^{T} X $ を密度行列 $ \displaystyle  \rho _{X} = \sum _{i,j} \left( X ^{T} X \right) _ {ij} \lvert i \rangle \langle j \rvert $ にエンコードする。

４）$ \displaystyle \frac{1}{\sqrt{N}} \sum _ {i = 1 } ^ {N} \lvert x _{i} \rangle \lvert i \rangle $ に対して制御 $ e^{- i \rho _{X} t} $ ゲートを使った位相推定サブルーチンを実行し、 $ X ^{T} X $ の固有値 $ \sigma _{i} ^ 2 $ を補助量子ビットに書き込む。 $ e^{- i \rho _{X} t} $ ゲートは密度行列冪によって SWAP ゲートで近似する。【3.7 p.77】

５）制御回転ゲートと補助量子ビットの測定　【2.7.2 p.48 振幅エンコーディング】により、$ \displaystyle \frac{1}{\sigma _{i} ^{2}} $ を掛け、$ \displaystyle \sum _{i=1} ^{r} \frac{1}{\sigma_{r}} \lvert v_r \rangle \lvert u_r \rangle $ を得る。

逆演算（uncomputation）で不要な量子ビットを $ \lvert 0 \rangle $ に戻す。


６） $ \displaystyle \lvert \psi _{1} \rangle =  \frac{1}{\sqrt{\sum_{k} \sigma _{k} ^{-2} }}   \sum _{k=1} ^{r} \frac{1}{\sigma _{k}} \lvert v_k \rangle \lvert u_k \rangle $ を得る。


（予測）

１）補助量子ビットが $ \lvert 1 \rangle _{a}$ のときに、未知の入力データ $ \tilde{x} $ と訓練出力 $ \mathbf{y} $ をエンコードした $ \displaystyle \lvert \psi _{2} \rangle = \lvert \tilde{x} \rangle \lvert y \rangle = \sum _{i=1} ^{N} \tilde{x_{i}} \lvert i \rangle \sum _{j=1} ^{M} \tilde{y_{j}} \lvert j \rangle $ を用意する。 

２）$\displaystyle \frac{1}{\sqrt{2}} \left( \lvert 0 \rangle _{a} \lvert \psi _1 \rangle + \lvert 1 \rangle _{a} \lvert \psi _2 \rangle \right) $ を準備し、補助量子ビットにアダマールゲートを作用させてから測定する。

３）測定結果が０となる期待値から内積 $ \langle \psi _1 \vert \psi _2 \rangle = f _{w^{*}} ( \tilde{x} ) $ が得られる。


[arXiv:1601.07823 Prediction by linear regression on a quantum computer](https://arxiv.org/pdf/1601.07823v2.pdf)

### 4.4.5 量子サポートベクタマシン

$ F \alpha ^{\prime} = y ^{\prime} $

[arXiv:1307.0471 Quantum support vector machine for big data classification](https://arxiv.org/pdf/1307.0471.pdf)


### 4.4.6 量子主成分分析


[arXiv:0810.0312 On the relationship between continuousand discrete-time quantum walk](https://arxiv.org/pdf/0810.0312.pdf)